# PDF Processing and Summarization Assignment

In [20]:
import re
import nltk
import spacy
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from pymongo import MongoClient
import concurrent.futures
from PyPDF2 import PdfReader
import PyPDF2
import os
from pymongo import MongoClient
from time import time
nlp = spacy.load('en_core_web_sm')
from pymongo import MongoClient

In [30]:
# Connect to the local MongoDB instance
# Create/use a database called 'pdf_summaries'
# Create/use a collection called 'documents'
# Confirm connection

In [21]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')

db = client['pdf_summaries']

collection = db['documents']

print("Connected to local MongoDB and using 'pdf_summaries' database.")

Connected to local MongoDB and using 'pdf_summaries' database.


In [ ]:
# Path to your folder containing the PDFs (replace with your actual folder path)
# List all PDF files in the folder
# Display the files

In [22]:
import os
folder_path = "C:\\Users\\saras\\Desktop\\wasserstoff\\PDF"
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
print("PDF Files:", pdf_files, )

PDF Files: ['1292585113260.pdf', '20240716890312078.pdf', '250883_english_01042024.pdf', 'admin_judgement_file_judgement_pdf_1950_volume 1_Part I_Commissioner of Income Tax, West Bengal_Calcutta Agency Ltd._1697606310.pdf', 'admin_judgement_file_judgement_pdf_1952_volume 1_Part I_the state of bihar_maharajadhiraja sir kameshwar singh of darbhanga and others_1698318448.pdf', 'admin_judgement_file_judgement_pdf_1960_volume 1_Part I_1699521350.pdf', 'admin_judgement_file_judgement_pdf_1960_volume 1_Part I_1699526804.pdf', 'admin_judgement_file_judgement_pdf_1960_volume 1_Part I_great indian motor works ltd., and another_their employees and others_1699336355.pdf', 'admin_judgement_file_judgement_pdf_1960_volume 1_Part I_messrs. ispahani ltd. calcutta_ispahani employees  union_1699338549.pdf', 'admin_judgement_file_judgement_pdf_1960_volume 1_Part I_phulbari tea estate_its workmen_1699339226.pdf', 'admin_judgement_file_judgement_pdf_1960_volume 1_Part I_shri b. p. hira, works manager, centr

In [ ]:
# Function to extract text from the PDF using PdfReader
# Function to process a single PDF (extract text and store in MongoDB)
# Extract text from the PDF
# Prepare the document data to insert into MongoDB
# MongoDB connection setup
# Process all PDFs in parallel using ThreadPoolExecutour
# Calculate total time taken

In [23]:
import os
import concurrent.futures
from pymongo import MongoClient
from PyPDF2 import PdfReader  
import time

def extract_text_from_pdf(pdf_file_path):
    try:
        reader = PdfReader(pdf_file_path)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_file_path}: {e}")
        return None

def process_pdf(pdf_file_name):
    try:
        pdf_file_path = os.path.join(folder_path, pdf_file_name)

        extracted_text = extract_text_from_pdf(pdf_file_path)

        if not extracted_text:
            print(f"Skipping {pdf_file_name}: No text extracted.")
            return

        document_data = {
            "file_name": pdf_file_name,
            "path": pdf_file_path,
            "size": f"{os.path.getsize(pdf_file_path) / 1024:.2f} KB",  # Size in KB
            "text": extracted_text
        }

        result = collection.insert_one(document_data)

        print(f"Processed and inserted {pdf_file_name} into MongoDB (ID: {result.inserted_id}).")
    
    except Exception as e:
        print(f"Error processing {pdf_file_name}: {e}")

folder_path = r'C:\Users\saras\Desktop\wasserstoff\PDF'  

client = MongoClient('mongodb://localhost:27017/')
db = client['pdf_summaries']
collection = db['documents']

pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

start_time = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_pdf, pdf_files)

end_time = time.time()
time_taken = end_time - start_time

print(f"Processed {len(pdf_files)} PDFs in {time_taken:.2f} seconds.")


Processed and inserted admin_judgement_file_judgement_pdf_1960_volume 1_Part I_1699526804.pdf into MongoDB (ID: 671a25aee6931cd4b07b2a85).
Processed and inserted admin_judgement_file_judgement_pdf_1960_volume 1_Part I_phulbari tea estate_its workmen_1699339226.pdf into MongoDB (ID: 671a25afe6931cd4b07b2a86).
Processed and inserted admin_judgement_file_judgement_pdf_1960_volume 1_Part I_1699521350.pdf into MongoDB (ID: 671a25b4e6931cd4b07b2a87).
Processed and inserted admin_judgement_file_judgement_pdf_1960_volume 1_Part I_shri b. p. hira, works manager, central railway, parel, bombay etc._shri c. m. pradhan etc._1699527217.pdf into MongoDB (ID: 671a25b5e6931cd4b07b2a88).
Processed and inserted admin_judgement_file_judgement_pdf_1960_volume 1_Part I_messrs. ispahani ltd. calcutta_ispahani employees  union_1699338549.pdf into MongoDB (ID: 671a25b6e6931cd4b07b2a8a).
Processed and inserted admin_judgement_file_judgement_pdf_1960_volume 1_Part I_the graham trading co. (india) ltd._its workm

In [24]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [25]:
import re
import nltk
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from nltk.tokenize import sent_tokenize
from pymongo import MongoClient

In [35]:
# Initialize the lemmatizer and stopwords
# Preprocess the text
# in this process i did make a connection with the mongo db to summarize the text 
# then by stop words, td-idf, frequency based sentence scoring, # Position-based sentence weighting, # TextRank-based sentence scoring
# i summarize the text and got the keywords 

In [27]:



lemmatizer = nltk.WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))
custom_legal_stopwords = ["whereas", "hereinafter", "furthermore", "therefore", "court", "judge", "upon", "hereby"]


def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = text.split()
    
    # Remove stopwords 
    words = [word for word in words if word not in stop_words and word not in custom_legal_stopwords]
    
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

# Frequency-based sentence scoring
def score_sentences_by_frequency(cleaned_text, sentences):
    word_frequencies = Counter(cleaned_text.split())
    sentence_scores = {}
    
    for sentence in sentences:
        sentence_clean = clean_text(sentence)
        score = sum(word_frequencies[word] for word in sentence_clean.split() if word in word_frequencies)
        sentence_scores[sentence] = score
        
    return sentence_scores

# Position-based sentence weighting 
def boost_sentence_position(sentence_scores, sentences):
    total_sentences = len(sentences)
    
    for i, sentence in enumerate(sentences):
        if i < total_sentences * 0.1:  
            sentence_scores[sentence] += 1
        elif i > total_sentences * 0.9:  
            sentence_scores[sentence] += 1
            
    return sentence_scores

# TextRank-based sentence scoring
def textrank_summarization(sentences, num_sentences=5):
    tfidf = TfidfVectorizer().fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf)
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summary = ' '.join([ranked_sentences[i][1] for i in range(num_sentences)])
    return summary

# Dynamic summary length based on document size
def determine_summary_length(text):
    word_count = len(text.split())
    if word_count <= 1000:
        return 3
    elif word_count <= 5000:
        return 5
    else:
        return 7

# Main summarization function
def summarize_legal_text(text):
    # Tokenize into sentences
    sentences = sent_tokenize(text)

    cleaned_text = clean_text(text)

    sentence_scores = score_sentences_by_frequency(cleaned_text, sentences)
     
    sentence_scores = boost_sentence_position(sentence_scores, sentences)

    ranked_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    
    num_sentences = determine_summary_length(text)
 
    summary_sentences = ranked_sentences[:num_sentences]

    summary = ' '.join(summary_sentences)
    
    return summary

# Function to extract keywords using TF-IDF
def extract_keywords(text, num_keywords=5):
    try:
        tfidf = TfidfVectorizer(max_features=num_keywords, stop_words='english') 
        tfidf_matrix = tfidf.fit_transform([clean_text(text)])
        keywords = tfidf.get_feature_names_out()
        return list(keywords)  
    except Exception as e:
        print(f"Error during keyword extraction: {e}")
        return []

# Function to fetch, summarize, extract keywords, and update MongoDB
def update_with_summary_and_keywords():
    documents = collection.find({"summary": {"$exists": False}})

    for document in documents:
        pdf_file_name = document["file_name"]
        text = document.get("text", "")  

        if not text:
            print(f"Skipping {pdf_file_name}: 'text' field not found or empty.")
            continue

        
        summary = summarize_legal_text(text)
        keywords = extract_keywords(text)

        print(f"Summary for {pdf_file_name}: {summary}")
        print(f"Keywords for {pdf_file_name}: {keywords}")

        collection.update_one(
            {"_id": document["_id"]},  
            {"$set": {"summary": summary, "keywords": keywords}}
        )

        print(f"Updated document {pdf_file_name} with summary and keywords.")

client = MongoClient('mongodb://localhost:27017/')
db = client['pdf_summaries']
collection = db['documents']

update_with_summary_and_keywords()


Summary for admin_judgement_file_judgement_pdf_1960_volume 1_Part I_1699526804.pdf: In the present case the jury had been clearly directed 
by the Sessions Judge that corroborative evidence 
must be evidence which implicates the accused, i.e., 
which confirms in some material particulars not only 
the evidence that the crime had been committed but 
also that the appellant had committed it. It would be 
in high degree dangerous to attempt to formulate 
the kind of evidence which would be regarded as 
corroboration, except to say that corroborative 
evidence is evidence which shows or tends to show 
that the story of the accomplice that the accused 
committed the crime is true, not merely that the 
crime has been committed, but that it was commit­
ted by the accused. 
" 953, that the approver's 
evidence had to satisfy a double test i.e., he must be a reliable 
witness and his evidence must receive sufficient corroboration and 
in that the corroborating evidence was not sufficient to con

Summary for admin_judgement_file_judgement_pdf_1960_volume 1_Part I_the graham trading co. (india) ltd._its workmen_1699523577.pdf: In dealing with puja bonus based on an 
'implied term of employment, it was pointed out by us 
in Messrs. lspahani Ltd. v. lspahani Employees' 
Union (1) that a term may be implied, even though the 
payment may not have been at a uniform rate 
throughout and the Industrial Tribunal would be 
justified in deciding what should be the quantum of 
payment in a particular year taking into account the 
varying payments made in pr1;1vious years. In determining whether the payment was 
customary and traditional the following circumstances have to be 
established : 
(i) that the payment has been made over an unbroken series 
of years; , 
(ii) that it has been for a sufficiently long period, the 
period has to be longer than in the case of an implied term of 
employment; 
(iii) that it has been paid even in years of loss and did not 
depend on the earning of profits

Summary for admin_judgement_file_judgement_pdf_1960_volume 1_Part I_the tata oil mills co. ltd._its workmen and others_1699333862.pdf: In resisting the workmen's claim for bonus for the year 
1955-56 the appellant contended that in calculating gross profits 
for the purpose of the Full Bench formula the following items of 
income should be excluded :-
(i) Income earned by way of rent, light and power ; 
(ii) estate revenue derived from sale of excess coconuts used 
in preparing oil grown in the appellant's groves; 
(iii) profit from sale of empty barrels ; and 
(iv) sale proceeds of tin cans, scraps, Jogs, planks, gunnies 
etc. There is no 
doubt that there must be contribution of the workmen 
in earning profits before they are entitled to profit 
bonus; but it wa.s not laid down in the Muir Mills \ Case(') that direct connection between the efforts of the 
workmen and the particular item of profit earned must 
be established before the profit can be taken into 
account for the purpose

Summary for 1292585113260.pdf: The order log and trade 
log revealed that there was synchronization of buying and selling orders  
by these brokers as under:  
 
Date  Trade 
Time  Tr 
No Order NO  Order time  Diff  
(sec)  B/S Mem  Client Code  Qty  Price  
27-02-01 12:16:13  1 61175322  12:16:13  **** buy 35 rathyatrasa  10000  335.00  
27-02-01 12:16:13  1 31317843  12:15:37  0:00:36  sell 545 b110  10000  335.00  
27-02-01 12:16:56  2 61175333  12:16:56  **** buy 35 rathyatrasa  10000 335.00  
27-02-01 12:16:56  2 31317859  12:16:32  0:00:24  sell 545 b110  10000  335.00  
27-02-01 12:17:18  3 61175338  12:17:18  **** buy 35 rathyatrasa  10000  335.00  
27-02-01 12:17:18  3 31317864  12:17:07  0:00:11  sell 545 b110  10000  335.00  
27-02-01 12:17:27  4 61175342  12:17:27  **** buy 35 rathyatrasa  10000  335.00  
27-02-01 12:17:27  4 31317867  12:17:17  0:00:10  sell 545 b110  10000  335.00  
27-02-01 12:17:33  5 61175347  12:17:33  **** buy 35 rathyatrasa  10000  335.00  
27-02-01

Summary for Circular Orders (Supplement).pdf: (C) SUMMARY TRIAL CASES: -  
(A) Appealable cases   
(i) Warrant cases  6 cases per day  
(ii) Summons cases:  6 cases per day  
(B) Non appealable cases:   
(i) Warrant cases  12 cases per day  
(ii) Summons cases  12 cases per day  
(C) Cases where the accused pleads guilty 
at the commencement of the trial. (A) WARRANT CASES: - 
(I) Cases  in which the accused is convicted 
or acquitted after full trial under Section 
363, 377, 408, 409, 467, 468, 471, 477 
and 277A, IPC  2 days for one case  
(II) Cases in which the accused is convicted or acquitted after full trial:  
(a) Essential Commodities Act/ The Food 
Safety and Standards Act, 2006  2 cases per day  
(b) Arms Act  3 cases per day  
(c) Prevention of Food adulteration Act  2 cases per day  
(d) Excise Act  4 cases per day  
(e) Cases triable by Spl. The Station House Officer of the concerned Police 
Station shall ensure that each warrant of arrest issued by the Court, when 
recei

Summary for 250883_english_01042024.pdf: 326.Whoever commits mischief by,—
(a) doing any act which causes, or which he knows to be likely to cause, a
diminution of the supply of water for agricultural purposes, or for food or drink for
human beings or for animals which are property, or for cleanliness or for carrying on
any manufacture, shall be punished with imprisonment of either description for a term
which may extend to five years, or with fine, or with both;
(b) doing any act which renders or which he knows to be likely to render any
public road, bridge, navigable river or navigable channel, natural or artificial, impassable
or less safe for travelling or conveying property, shall be punished with imprisonment
of either description for a term which may extend to five years, or with fine, or with
both;
(c) doing any act which causes or which he knows to be likely to cause an
inundation or an obstruction to any public drainage attended with injury or damage,
shall be punished with i

Summary for admin_judgement_file_judgement_pdf_1952_volume 1_Part I_the state of bihar_maharajadhiraja sir kameshwar singh of darbhanga and others_1698318448.pdf: We find that under article 246 Parliament 
has exclusive power to make laws with respect, inter 
ulia, to matters specified in entry 33 in List I, namely, 
"acquisition or requisitioning of property for the pur-
poses of the Union, that the State Legislatures have 
exclusive power to make laws with respect, inter 
alia, to matters specified in entry 36 in List II, namely, 
the "acquisition or requisitioning of property except 
for the purposes of the Union subject to the provision 
of entry 42 of List III" and that both Parliament and 
the State Legislatures may make laws with respect to 
matters set forth in entry 42 in List III, namely, the 
principles for determining the compensation and the 
form and manner of giving such compensation. If the require-
ment of a public purpose were not a provision of 
article 31 (2), then 

In [32]:
# Full path of the PDF file in the database
# Find the document by file path
# just to see its working i see one pdf and its summary and keywords 

In [29]:

pdf_file_path = r'C:\Users\saras\Desktop\wasserstoff\PDF\admin_judgement_file_judgement_pdf_1950_volume 1_Part I_Commissioner of Income Tax, West Bengal_Calcutta Agency Ltd._1697606310.pdf'  # Use the actual full file path

document = collection.find_one({"path": pdf_file_path}, {"file_name": 1, "text": 1, "summary": 1, "keywords": 1})

if document:
    print(f"\n===== File Name =====\n{document['file_name']}\n")
    
    print(f"===== Original Text (Glimpse) =====\n{document['text'][:500]}...\n")  
    
    print(f"===== Shortened Summary =====\n{document['summary']}\n")
    
    print(f"===== Keywords =====\n{', '.join(document['keywords'])}\n")
else:
    print(f"No document found with file path: {pdf_file_path}")



===== File Name =====
admin_judgement_file_judgement_pdf_1950_volume 1_Part I_Commissioner of Income Tax, West Bengal_Calcutta Agency Ltd._1697606310.pdf

===== Original Text (Glimpse) =====
1950 
Patntalal 
Jankid-:u 
v, 
Mohanlal and 
Another, 
Pata1tjali 
S11stri J. 
!950 
Deo, 21. 1008 SUPREME COURT REPORTS [1950] 
of section 14, it seems to me, they would be bringing 
themselves under the bar of section 18 (2). The 
respondents cannot therefore claim that the loss of the 
goods was explosion damage within the meaning of the 
Ordinance so as to bring the case within section 14 and 
at the same time contend that the loss was not "due 
to or did not in any way arise ont of the exp...

===== Shortened Summary =====
Mitter 
or S.C. Mitter, having been and being still the Manag-
ing Agents of the said Mills have undertaken to 
reimburse the said Mills in respect of the decrees to oe 
made in the said four suits in the manner hereinafter 
appearing NOW THESE PRESENTS WITNESS 
AND IT IS 